In [ ]:
#this creates a csv file 'universityList.csv' containing name, key and link for all university.
# Note to self
# ------- Do not Execute this cell -----------

getUniversityList()

In [63]:
import requests
import sys
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
from requests.exceptions import HTTPError
import pandas as pd
import numpy as np
import time

In [64]:
def open_url_2_soup(url):
    """
    open webpage and convert to beautiful soup object
    returns: page_soup(soup object)
    params: url(str) - url of the page
    
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    page = requests.get(url,headers = headers,verify = False, allow_redirects = False)
    if(page.status_code == 200):
        page_soup = soup(page.content,'lxml')
        return page_soup
    else: 
        raise HTTPError()

In [3]:
def getUniversityLookup(url):
    """
    get the list of university and it's basic info from ONE page
    returns : univ_df - dataframe containing name,key and url of universities one page at a time
    params: url - url of the page
    
    """
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append("https://yocket.in"+a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df

In [4]:
def getUniversityList():
    """
    creates a csv file of all university detail
    returns: UniversityList_df - dataframe containg the basic details of top 500 engineering schools
    params: None
    
    """
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(5) 
        count += 1
    UniversityList_df.to_csv('universityList.csv', encoding='utf-8')
    return UniversityList_df


In [5]:

def getUniversityDetail(url):
    """
    get the details of a single university
    params: url(string) - url of the site
    returns : univ_detail(dict) - dictionary containing details of a single university
    
    """
    univ_detail = {}
    page_soup = open_url_2_soup(url)
    univ_tag = page_soup.findAll("div",{"class":"col-sm-3 col-xs-6"})
    univ_detail['name'] = page_soup.findAll("h1")[0].text
    tag_no = 1;
    for tag in univ_tag:
        if(tag_no in [1,2,3,4,6,8,9]):
            name,data = extractUnivDetails(tag,tag_no)
            univ_detail[name] = data
        tag_no+= 1
    try:
        univ_detail['admitList'] = univ_detail['student_link']+"2"
        univ_detail['rejectList'] = univ_detail['student_link']+"3"
        univ_detail['appliedList'] = univ_detail['student_link']+"1"
        del univ_detail['student_link']
    except:
        univ_detail['admitList'] = None
        univ_detail['rejectList'] = None
        univ_detail['appliedList'] = None
        del univ_detail['student_link']
    return(univ_detail)
# getUniversityDetail("https://yocket.in/university-reviews/university-of-kentucky-2619/engineering")

In [6]:
def extractUnivDetails(tag,tag_no):
    """
    returns : univ_dict(dictionary) - dictionary containing university(single) detail
    params: tag(soup element) - the div containing the element
            tag_no(int) - number of div
    """
    univ_dict = {}
    if(tag_no in [1,2,8,9]):
        return(tag.h3.small.text,(tag.br.previous_sibling)[1:])
    elif(tag_no ==3):
        return (tag.h3.findAll("small")[1].text,tag.h3.find("small").previous_sibling)
    elif(tag_no ==4):
        return (tag.h3.findAll("small")[1].text,(tag.h3.find("small").previous_sibling)[2:])
    elif(tag_no == 6):
        try:
            a = tag.findAll("a",href=True)[0]
        except:
            return("student_link",None)
        else:
            return("student_link",("https://yocket.in"+a['href'])[:-1])


In [7]:
def univDetail_to_csv():
    """
    generate csv file containing university details
    params : None
    returns : None
    """
    univ_list = pd.read_csv('universityList.csv')
    universityDetail = pd.DataFrame()
    counter = 51
    while(counter <= 100):
        link = univ_list.iloc[counter]['href']
        dict_temp = getUniversityDetail(link)
        universityDetail = universityDetail.append(dict_temp,ignore_index = True)
        print(counter)
        counter += 1
        time.sleep(10)
#     print(universityDetail)
    universityDetail.to_csv('universityDetailList.csv', encoding='utf-8',header = False,mode = 'a')

In [ ]:
#create the cab file of university list with all it's parameter
#  Note to self
# -----------DO NOT EXECUTE THIS CELL --------------
univDetail_to_csv()   

In [48]:
def getStdList(page_soup,admit_status):
    name_tag = page_soup.findAll("input" , {"id" : "users-view-search-universities"})[0]
    name = name_tag['value']
    grading = page_soup.findAll("div",{"class" : "col-sm-6"})[1:]
#     print(grading[0])
    student_df = pd.DataFrame()
    for tag in grading:
#         print(tag)
        std_data = {}
        std_data['college'] = name
        std_data['admit/reject'] = admit_status
        a = tag.findAll("a",href = True)[0]
#         print(a)
        std_data['Name'] = a.text #Tag for name of student
#         std_data['studentLink'] - a['href']
        std_data['CourseName'] = tag.find("small").text #Tag for course
        student_data_tag = tag.findAll("div",{"class":"col-sm-3 col-xs-6"})
        for data_tag in student_data_tag:
            heading = data_tag.find("strong").text
            data = data_tag.findNext("br").next_sibling
            if (heading == "GRE"):
                std_data['GRE'] = data.string

            elif(heading == "TOEFL" or heading == "IELTS" or heading == "ENG TEST"):
                std_data['ENG_TEST'] = data.string

            elif("UNDERGRAD"):
                std_data['CGPA']= data.string

            elif("WORK EX"):
                std_data['WorkExp'] = data.string
#         print(std_data)
        student_df = student_df.append(std_data, ignore_index=True)
#     print(student_df)
    return student_df


In [71]:
def getStudentDetailList(url):
    page_counter = 1
    student_detail = pd.DataFrame()
    url = url + "?page=4"
    while(True):
        
        print(url)
        try:
            page_soup = open_url_2_soup(url)
        except HTTPError:
            print("exception raised")
            page_counter = 1
            break
        else:
            print("in else")
            std_detail_page = getStdList(page_soup,'admit')
            student_detail = pd.concat([student_detail,std_detail_page])
            page_counter += 1
            url = url[:-1] + str(page_counter)
        
#     print(student_detail.head(4))
        

In [73]:
u = "https://yocket.in/applications-admits-rejects/50461-eindhoven-university-of-technology/2"
# page_soup = open_url_2_soup(u)
# grading = page_soup.findAll("div",{"class" : "col-sm-6"})[]
# print(grading)
getStudentDetailList(u) 
#name
# print(data['value']) #name


https://yocket.in/applications-admits-rejects/50461-eindhoven-university-of-technology/2?page=4
exception raised


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [62]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
url = "https://yocket.in/applications-admits-rejects/50461-eindhoven-university-of-technology/2?page=2"
request = requests.get(url , headers = headers, verify = False, allow_redirects=False)
if request.status_code == 200:
    print('Web site exists')
else:
    print('Web site does not exist')

/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Web site does not exist
